In [ ]:
import json
import requests
import random
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
import seaborn as sns
from textwrap import wrap
import squarify
import matplotlib.ticker as mtick
import plotly.express as px

import of data containing the id of the film by which we will draw films

In [ ]:
title_akas = data=pd.read_csv('https://datasets.imdbws.com/title.akas.tsv.gz',sep='\t',
                              usecols = ['titleId', 'isOriginalTitle'])
title_akas

select only unique titles

In [ ]:
title_akas_sorted = title_akas[title_akas['isOriginalTitle'] == '1']
title_akas_sorted

transfer id to the list

In [ ]:
Id_list = title_akas_sorted['titleId'].values.tolist()
Id_list[:10]

we choose the seed to get the same films, in addition we divide the download into two days because the daily free limit is 1000

In [ ]:
random.seed(10)

In [ ]:
sample_2k = random.sample(Id_list,2000)

In [ ]:
len(sample_2k)

In [ ]:
sample_2k1 = sample_2k[:1000]

In [ ]:
len(sample_2k1)

In [ ]:
sample_2k2 = sample_2k[1000:]

In [ ]:
len(sample_2k2)

In [ ]:
films = pd.DataFrame()

draw sample from sample_2k and next day from sample_2k2, be sure to put your apikey below

In [ ]:
for i in sample_2k2:
    url1 = f'http://www.omdbapi.com/?i={i}&apikey='    
    result = requests.get(url1).json()
    df = pd.DataFrame.from_dict(result, orient = 'index').T
    if df['Response'][0] == 'True' :
        films = films.append(df)
    else:
        pass
films     

In [ ]:
films.to_csv('films1k2.csv', sep = ',', encoding='utf-8', index = False )

Set your path

In [ ]:
path_B = ''
# path_X = ''

In [ ]:
films1k = pd.read_csv(path_B +'films1k.csv')
films1k2 = pd.read_csv(path_B+'films1k2.csv')

In [ ]:
films1k

In [ ]:
films1k
del films1k['Unnamed: 0']

In [ ]:
df = films1k.append(films1k2)

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df.head()

descriptive statistics

In [ ]:
df["imdbVotes"] = df["imdbVotes"].str.replace(',','.')

In [ ]:
df['imdbVotes']=df['imdbVotes'].astype(float)

In [ ]:
df['imdbVotes'].describe()

In [ ]:
df['imdbRating'].describe()

Save appended df

In [ ]:
df.to_csv('films_appended.csv', sep = ',', encoding='utf-8', index = False )

In [ ]:
df.columns

# Reading preapared df

In [ ]:
# if needed
df = pd.read_csv(path_B + 'films_appended.csv')

### 1 figure (most common genres )

In [ ]:
df['Genre']

In [ ]:
temp=df['Genre'].value_counts()
temp

In [ ]:
temp.head(20)

In [ ]:
y = df['Genre'].value_counts().head(25)
    
plt.rcParams['figure.figsize'] = (35, 20)
plt.style.use('bmh')

labels = [ '\n'.join(wrap(l, 20)) for l in y.index]

color = plt.cm.twilight(np.linspace(0, 1,15))
squarify.plot(sizes = y.values, label = labels, alpha=.8, color = color, text_kwargs={'fontsize': 15})
plt.title('20 najczęściej wybieranych gatunków filmowych', fontweight = 30, fontsize = 30)

plt.axis('off')
plt.show()

### 2 figure (number of films per year)

In [ ]:
temp1 = df[df["Year"].str.contains("–")==False]

In [ ]:
temp1 = temp1[temp1["Year"].str.contains("2023|2022|2021|2020|2019|2018|2017|2016|2015|2014")==True]

In [ ]:
temp1=temp1.sort_values(by=['Year'])

In [ ]:
fig,ax = plt.subplots(figsize=(15,6))
sns.countplot(x='Year', 
              data=temp1, 
              palette='bright', 
              order = temp1['Year'].unique()
               )
ax.set_title('Ilość produkcji filmów na przestrzeni lat', fontweight = 30, fontsize = 20)
ax.set_xlabel("")
ax.set_ylabel("Liczba filmów", fontsize = 12)
ax.bar_label(ax.containers[0])

### 3 figure (most frequent countries)

In [ ]:
df1 = df.copy()

In [ ]:
df1['Country'] = df1['Country'].replace(['United States'], 'USA')

In [ ]:
df1['Country'] = df1['Country'].replace(['United Kingdom'], 'UK')

In [ ]:
dfk1= df1['Country']

In [ ]:
dfk1

In [ ]:
tempdfk1 = df1['Country'].value_counts()

In [ ]:
tempdfk1[0:10]

In [ ]:
data = {
    "Country" : ["USA", "UK","Germany","France","Spain","Italy","Japan","Australia","Brazil","Mexico"],
    'Count': list(tempdfk1[0:10])
}

In [ ]:
plot_df = pd.DataFrame(data)

In [ ]:
plot_df

In [ ]:
df2=df1.loc[df1['Country'].isin(["USA", "UK","Germany","France","Spain","Italy","Japan","Australia","Brazil","Mexico"])]

In [ ]:
fig_dims = (15,6)
fig, ax = plt.subplots(figsize=fig_dims)
sns.barplot(y=plot_df["Country"], order = plot_df.sort_values('Count', ascending = False).Country,
            x=plot_df["Count"], ax=ax, color="orange")

ax.set_title("Podział ze względu na kraje", fontweight = 30, fontsize = 20)
ax.set_xlabel("Liczba filmów")
ax.set_ylabel("")
sns.despine()

In [ ]:
fig_dims = (15,6)
fig, ax = plt.subplots(figsize=fig_dims)
sns.barplot(y=df2["Country"].value_counts().index, order = tempdfk1.sort_values('Country'),
            x=df2["Country"].value_counts(), ax=ax, color="orange")

ax.set_title("Podział ze względu na kraje", fontweight = 30, fontsize = 20)
ax.set_xlabel("Liczba filmów")
ax.set_ylabel("")
sns.despine()

### 4 figure (most frequent languages)

In [ ]:
df1['Language']

In [ ]:
df2=df1['Language'].value_counts()

In [ ]:
d1 = df2.head(30)

In [ ]:
d1.index

In [ ]:
def bbar(data, textdist):

    ax=data.plot.bar(color='indigo')
    
    ymin, ymax = plt.ylim()
    plt.ylim(ymin, ymax + 0.05 * (ymax - ymin))

    
    totals=[]
    for i in ax.patches:
        totals.append(i.get_height())

    total = sum(totals)
    
    for i in ax.patches:
            ax.text(i.get_x(), i.get_height()+textdist, str(round((i.get_height()/total)*100, 2))+'%', fontsize=10, color='black')

In [ ]:
df2=df1.loc[df1['Language'].isin(['English', 'Spanish', 'German', 'French', 'Japanese', 'Italian',
       'Portuguese',  'Russian', 'English', 'Greek', 'Danish'])]

In [ ]:
fig, ax = plt.subplots(figsize=(15,6))
plt.title('Podział ze względu na języki', fontweight = 30, fontsize = 20)
plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1000.0))
bbar(df2['Language'].value_counts(), 25,)

### 5 figure (countries where there is a nomination or award)

In [ ]:
df1['Country'] = df1['Country'].replace(['United States'], 'USA')

In [ ]:
df1['Country'] = df1['Country'].replace(['United Kingdom'], 'UK')

In [ ]:
df1['Awards'].dropna()

In [ ]:
df2=df1.loc[df1['Awards'].notna()]

In [ ]:
df3=df2['Country'].value_counts()

In [ ]:
data = {
    "Country" : ["USA", "UK","Germany","Japan","Spain","Brazil","Finland","Canada","South Korea","France"],
    'Count': list(df3.head(10))
}

In [ ]:
df4 = pd.DataFrame(data)

In [ ]:
df4

In [ ]:
df3=df2.loc[df2['Country'].isin(["USA", "UK","Germany","Japan","Spain","Brazil","Finland","Canada","South Korea","France"])]

In [ ]:
fig,ax = plt.subplots(figsize=(15,6))
sns.barplot(y=df4["Country"], x= df4['Count'], data=df4,palette='icefire')
ax.set_title('Ilość nominacji/nagród w podziale na kraje', fontweight = 30, fontsize = 20);
ax.set_xlabel("")
ax.set_ylabel("")

In [ ]:
temp2 = df[df["Year"].str.contains("–")==False]

In [ ]:
temp2 = temp2[temp2["Year"].str.contains("2023|2022|2021|2020|2019|2018|2017|2016|2015|2014|2013|2012|2011|2010|2009|2008|2007|2006|2005|2004|2003|2002|2001|200")==True]

### 6 figure of average film ratings over the years

In [ ]:
rates = temp2.groupby('Year')
rates2 = rates[['imdbRating']].mean()
rates2.dropna()

In [ ]:
fig = px.line(rates2, y="imdbRating", title='Oceny vs lata')
fig.show()

In [ ]:
data1 = df.groupby('Director')
max_rating = data1.imdbRating.mean()
max_rating.sort_values(ascending = False)